## Урок 5. Домашнее задание

1. Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

Загрузим необходимые библиотеки:

In [404]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

from lightfm import LightFM

import os, sys
sys.path.insert(1, os.getcwd() + '/webinar_5')
    
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

Загрузим данные, проведем необходимые начальные преобразования, а также разбиение на обучающий и тестовый датасеты:

In [405]:
data = pd.read_csv('webinar_5/data/retail_train.csv')

item_features = pd.read_csv('webinar_5/data/product.csv')
user_features = pd.read_csv('webinar_5/data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Создадим датафрейм на основе тестовых данных для оценки результатов модели:

In [406]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Проведем префильтрацию обучающего датасета, используя функцию из модуля `utils`:

In [407]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/rk/Documents/Recommender_systems/L5/webinar_5/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


Создадим user_item матрицу и необходимые сопоставления бизнес id пользователей и товаров с матричным порядком:

In [408]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='sum', # Выберем такую агрегирующую функцию для улучшения метрик
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# sales = data_train_filtered.groupby('item_id')['sales_value'].sum().reset_index()
# sales.rename(columns={'sales_value': 'sold'}, inplace=True)

# total_sum = np.sum(sales['sold'])

# sales['weight'] = sales['sold'] / total_sum

# for column in user_item_matrix.columns.to_list():
#     user_item_matrix.loc[:, column] *= sales.loc[sales['item_id'] == column, 'weight'].item()
    
# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(10)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [409]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Переработаем загруженные данные по пользователям и товарам в необходимый для дальнейшего использования формат.

In [410]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [411]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


Создадим бинарные признаки на основе категориальных признаков, которые затем будут использоваться в модели:

In [412]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [413]:
no_components = [20, 40, 80]
losses = ['bpr', 'warp']
learning_rate = [0.005, 0.01, 0.02]
item_alpha = [0.2, 0.4, 0.6]
user_alpha = [0.05, 0.1, 0.2]
k_values = [2, 5, 10, 15] ### не удалось обнаружить серьезное влияние
n_values = [10, 15, 25] ### не удалось обнаружить серьезное влияние
max_sampled_values = [50, 100, 150] ### не удалось обнаружить серьезное влияние
epochs = [10, 20, 30]

In [414]:
# подготавливаем id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

Проведем подбор гиперпараметров модели по сетке.

In [415]:
%%time

precision = 0
for comps in no_components:
    for loss in losses:
        for lr in learning_rate:
            for ia in item_alpha:
                for ua in user_alpha:
                    for epoch in epochs:
                        model = LightFM(no_components=comps,
                                        loss=loss,
                                        learning_rate=lr, 
                                        item_alpha=ia,
                                        user_alpha=ua, 
                                        random_state=42,
                                        k=5,
                                        n=15,
                                        max_sampled=100)
                        model.fit((sparse_user_item > 0) * 1,
                                  sample_weight=coo_matrix(user_item_matrix),
                                  user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                  item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                  epochs=epoch, 
                                  num_threads=20,
                                  verbose=False)
                        predictions = model.predict(user_ids=users_ids_row,
                                                    item_ids=items_ids_row,
                                                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                                    num_threads=10)
                        data_train_filtered['score'] = predictions
                        predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
                            unique().reset_index()
                        
                        df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
                        precision_calc = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
                        if precision_calc > precision:
                            precision = precision_calc
                            print(f'Components - {comps}, Loss - {loss}, Learning_rate - {lr},'  
                              f'Item-alpha - {ia}, User-alpha - {ua}, '
                              f'Epochs - {epoch} -- precision: {precision}')

Components - 20, Loss - bpr, Learning_rate - 0.005,Item-alpha - 0.2, User-alpha - 0.05, Epochs - 10 -- precision: 0.1469293924466338
Components - 20, Loss - bpr, Learning_rate - 0.005,Item-alpha - 0.4, User-alpha - 0.05, Epochs - 30 -- precision: 0.15264367816091953
Components - 20, Loss - bpr, Learning_rate - 0.02,Item-alpha - 0.4, User-alpha - 0.05, Epochs - 30 -- precision: 0.15353037766830868
Components - 40, Loss - bpr, Learning_rate - 0.01,Item-alpha - 0.6, User-alpha - 0.1, Epochs - 10 -- precision: 0.1544170771756979
CPU times: user 6h 26min 9s, sys: 2min 25s, total: 6h 28min 34s
Wall time: 6h 38min 51s


Построим итоговую модель с отобранными гиперпараметрами.

In [420]:
model = LightFM(no_components=40,
                loss='bpr',
                learning_rate=0.01, 
                item_alpha=0.6,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)
model.fit((sparse_user_item > 0) * 1,
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=10, 
          num_threads=20,
          verbose=False)
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)
data_train_filtered['score'] = predictions
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
    unique().reset_index()

df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
precision_calc = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f'Precision - {precision_calc:6f}')

Precision - 0.154417
